# NATS simulation and result visualization using <code>para-atm</code>

This notebook demonstrates the process of running a generic NATS simulation for an existing set of TRX and MFL files using <code>para-atm</code>. This notebook is general and can be repurposed for any NATS input.

Some specific notes and requirements to consider when running the following code:
<ul>
    <li>The NATS input TRX and MFL files used could be generated from <code>nats-fpgen.ipynb</code> which can be found <a href=http://localhost:8888/notebooks/nats-fpgen.ipynb>here</a>. This will will create a NATS input file from a user-defined IFF file.</li>
    <li>Required packages: <code>os</code>, <code>time</code>, <code>jpype</code>, and <code>para-atm</code> which can be cloned from the <a href="https://github.com/ymlasu/para-atm">para-atm</a> respository. </li> 
    <li>In addition, setup of NATS and the <code>NATS_HOME</code> variable is required. Testing in <code>para-atm</code> is available for verifying the NATS setup and compatibility with <code>para-atm</code>.</li>
</ul>

## Step 1: Specify NATS input TRX and MFL files

First provide the location of the existing TRX and MFL files to be input in the NATS simulation. Here, the example files are in the "Current Working Directory (CWD)" and so the <code>os</code> package retrieves the current working directory to find the input files and write the NATS output.

The name of the TRX and MFL files used in this example are <code>iff_to_gnats_geo.trx</code> and <code>iff_to_gnats_geo_mfl.trx</code>, respectively. These two input files were generated using <code>nats-fpgen.ipynb</code> which is co-located in this repository. The NATS output file will be <code>iff_to_gnats_geo.csv</code>.


In [1]:
import os
trx_dir = os.getcwd()
results_dir = os.getcwd()

trx_fname = '/iff_to_gnats_geo_SFO_test2'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
coord_file = trx_dir+trx_fname+'.crd'

results_file = results_dir+trx_fname+'.csv'

## Step 2: Run NATS simulation

Next, import the <code>GateToGate</code> simulation class from the local <code>gnats_gate_to_gate.py</code> file which is modified from the test script found in the <code>para-atm</code> Python package. The modifications were such so that the only inputs to a simulation instance is the TRX and MFL files. The results are written to a csv file at the requested location.

In [2]:
from gnats_gate_to_gate import GateToGate

natsSim = GateToGate()
data=natsSim(trx_name=trx_file,mfl_name=mfl_file)['trajectory']
natsSim.write_output(results_file)

SFOlat,SFOlon = list(natsSim.airportInterface.getLocation('KSFO'))

natsSim.cleanup()

### Step 3: Shift simulation timestamps to be consistent with iff data


In [3]:
import pandas as pd

with open(coord_file, 'r') as rFile:
    rLines = rFile.readlines()
    
trxCallsigns = [line.split(' ')[0] for line in rLines]
trxTracktimes = [int(line.split(' ')[1]) for line in rLines]
trxLatitudes = [float(line.split(' ')[2]) for line in rLines]
trxLongitudes = [float(line.split(' ')[3]) for line in rLines]

trxTimestamps = [pd.Timestamp("1970-01-01") + pd.Timedelta(seconds=tracktime) for tracktime in trxTracktimes]

In [4]:
import numpy as np
import pandas as pd

dfSim = data[['time','latitude','longitude','heading','callsign']]

for no, cs in enumerate(trxCallsigns):
    
    df = dfSim[dfSim.callsign==cs]
                                                                                              
    trxLat, trxLon, trxTracktime = trxLatitudes[no], trxLongitudes[no], trxTracktimes[no] 
    simTimes = df['time']
    
    minDist, tStart = None, 0
    for tIncr, tVal in enumerate(simTimes):

        simLat, simLon = df['latitude'].iloc[tIncr], df['longitude'].iloc[tIncr]

        tDist = np.sqrt((trxLat-simLat)**2+(trxLon-simLon)**2)
        tDir = (trxLat-simLat)*(SFOlat-simLat)+ (trxLon-simLon)*(SFOlon-simLon)
        
        minDist= tDist if minDist is None else minDist          
        if tDist < minDist: minDist, tStart = tDist, tVal
    
    tStartSecs =(tStart-pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')    
    tOffset = trxTracktime - tStartSecs
 
    for tIncr, tVal in enumerate(simTimes):
        dfSim.loc[(dfSim['callsign']==cs) & (dfSim['time']==tVal), ['time']] = tVal +  pd.Timedelta(seconds=tOffset)
   

/home/ghaikal/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


## Step 4: Plot NATS simulation results

Upon successful execution of the simulation, use the <code>plot_trajectory</code> function within the <code>para-atm.plotting</code> library to plot the resulting flight plan trajectories from NATS. 

In this example, the results contain four detailed trajectories capturing ground operations (three departures and one arrival) around SFO. The focus here is to replicate aircraft landing and taking off at SFO.

In [5]:
from paraatm.plotting import plot_trajectory

plotTimeframe = 800

plotStart = pd.Timestamp("1970-01-01") + pd.Timedelta(seconds=trxTracktimes[0])
plotEnd = plotStart + pd.Timedelta(seconds=plotTimeframe)

dfPlot = dfSim.loc[dfSim['time']>=plotStart]

plot_trajectory(dfPlot, output_notebook=True, plot_width=600, plot_height=400)

2019-05-11 07:54:28 2019-05-11 08:07:48


Loading BokehJS ...

## Step 5: Compare NATS simulation results to IFF data
Likewise, the the <code>plot_trajectory</code> function within the <code>para-atm.plotting</code> library can plot IFF data (recType=3) as well.

First, because NATS changes the working directory behind the scenes, we must use the <code>os</code> package to navigate back to the directory with the IFF files. Then, we can plot the IFF data and study the simulation with the NATS simulation results.

In [7]:
import os
from paraatm.io.iff import read_iff_file

# get flight call signs from mfl file
with open(mfl_file, 'r') as fname:
    mflLines = fname.readlines()
callsigns = [key.split(' ')[0] for key in mflLines[1:]]

# plot iff data for selected flights
home=os.getenv('HOME')
iff_dir = home+'/para-atm-collection/miscellaneous/gnats-fpgen/'
iff_fname = iff_dir+'IFF_SFO+ASDEX_20190511_080104_86221.csv'

iff_data =  read_iff_file(iff_fname,record_types=[2,3,4,8])

plot_data = iff_data[3].loc[iff_data[3]['callsign'].isin(callsigns)]

plot_trajectory(plot_data, output_notebook=True, plot_width=600, plot_height=400)

Loading BokehJS ...

In [5]:
natsSim.terminalAreaInterface.getAllApproaches('KLAX')

<jpype._jarray.java.lang.String[] at 0x7fbbac363af0>

In [6]:
list(_)

['H06LZ',
 'H06RZ',
 'H07LZ',
 'H07RZ',
 'H24LZ',
 'H24RZ',
 'H25LZ',
 'H25RZ',
 'I06L',
 'I06R',
 'I07L',
 'I07R',
 'I24L',
 'I24R',
 'I25L',
 'I25R',
 'L06L',
 'L06R',
 'L07L',
 'L07R',
 'L24L',
 'L24R',
 'L25L',
 'L25R',
 'R06LY',
 'R06RY',
 'R07LY',
 'R07RY',
 'R24LY',
 'R24RY',
 'R25LY',
 'R25RY']